In [1]:
### Economic Development Capstone

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import hashlib
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import statistics

%matplotlib inline

pd.options.display.float_format = '{:.2f}'.format #Stop displaying scientific notation
pd.set_option('display.max_columns', None) #Display all columns

### Load Data:

In [7]:
df = pd.read_csv("ny_data.csv")

### Rename Columns

In [8]:
df.columns

Index(['Project\n ID #', 'Recipient Name', 'Project Name',
       'Project Description', 'Project Address', 'County', 'Postal Code',
       'Region', 'Industry', 'Start Date', 'End Date', 'Assistance Type',
       'Total ESD Assistance Awarded', 'Total Public-Private Investment',
       'Disbursements\n To-Date', 'Project Status', 'Compliant?',
       'Reason for Non-Compliant Status', 'Penalties Applied',
       'Rationale for Termination', 'Job Creation Commitments',
       'Job Retention Commitments', 'Jobs Created To-Date',
       'Jobs Retained \nTo-Date', 'Project Hires'],
      dtype='object')

In [9]:
#Rename columns
df.rename({'Project\n ID #': 'project_id', 'Recipient Name': 'company_name', 'Project Address':'company_address',
               'Project Name': 'project_name', 'Project Description':'project_description', 'County':'county',
              'Postal Code':'zip_code', 'Industry':'business_industry', 'Region':'region',
              'Start Date':'start_date','End Date':'end_date',
              'Assistance Type':'assistance_type', 'Total ESD Assistance Awarded':'total_esd_award',
              'Total Public-Private Investment': 'total_project_cost','Disbursements\n To-Date':'disbursements_to_date',
              'Compliant?':'compliant', 'Reason for Non-Compliant Status':'reason_for_noncompliance',
              'Penalties Applied':'penalties_applied',
              'Rationale for Termination':'rationale_for_term',
              'Job Creation Commitments':'projected_jobs_created',
              'Job Retention Commitments':'projected_jobs_retained',
              'Jobs Created To-Date':'final_jobs_created', 'Jobs Retained \nTo-Date':'final_jobs_retained',
              'Project Hires':'project_hires', 'Project Status':'status'
              }, axis=1, inplace=True)

In [10]:
df.head(5)

,project_id,company_name,project_name,project_description,company_address,county,zip_code,region,business_industry,start_date,end_date,assistance_type,total_esd_award,total_project_cost,disbursements_to_date,status,compliant,reason_for_noncompliance,penalties_applied,rationale_for_term,projected_jobs_created,projected_jobs_retained,final_jobs_created,final_jobs_retained,project_hires
0,ET1820,"Flextrapower, Inc. (internship training)","Flextrapower, Inc. (internship training)","Tax Credit, Health Care and Social Assistance,...",29-10 Thomson Ave,Queens,11101,New York City,Health Care and Social Assistance,05/01/2020,06/03/2021,Tax Credit,"6,000","19,650",6000,Closed,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ET1020,"Evisagenics, Inc. (internship training)","Evisagenics, Inc. (internship training)","Tax Credit, Health Care and Social Assistance,...",101 6th Ave,New York,10013,New York City,Health Care and Social Assistance,06/01/2020,04/20/2021,Tax Credit,"3,000","6,000",3000,Closed,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ET1520,Redesign Science Inc. (internship training),Redesign Science Inc. (internship training),"Tax Credit, Health Care and Social Assistance,...",180 Varick St,New York,10014,New York City,Health Care and Social Assistance,06/01/2020,04/20/2021,Tax Credit,"2,329","4,658",2329,Closed,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ET1620,"Vant AI, Inc. (internship training)","Vant AI, Inc. (internship training)","Tax Credit, Health Care and Social Assistance,...",33 Irving Place,New York,10003,New York City,Health Care and Social Assistance,06/01/2020,04/20/2021,Tax Credit,"15,000","45,851",15000,Closed,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ET920,Cureatr Inc. (internship training),Cureatr Inc. (internship training),"Tax Credit, Health Care and Social Assistance,...",17 West 20th St,New York,10011,New York City,Health Care and Social Assistance,06/01/2020,04/20/2021,Tax Credit,"6,000","13,240",6000,Closed,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
## Examining Status Column

Now we need to rename these various statuses so they match up with our Iowa data.

In [7]:
# Success = 2
# Cancelled = 0
# WIP = 1

df['status'].replace({'Active':'1'}, inplace=True)
df['status'].replace({'Compliance Complete':'2'}, inplace=True)
df['status'].replace({'Out of Business- written off':'0'}, inplace=True)
df['status'].replace({'Out of Business- confirmed':'0'}, inplace=True)
df['status'].replace({'Out of Business- Not written off':'0'}, inplace=True)
df['status'].replace({'Out of Business- not written off':'0'}, inplace=True)
df['status'].replace({'Repaid':'2'}, inplace=True)
df['status'].replace({'Out of Business':'0'}, inplace=True)
df['status'].replace({'Out of Business- Loan repaid in full':'2'}, inplace=True)
df['status'].replace({'Released':'2'}, inplace=True)
df['status'].replace({'OOB':'0'}, inplace=True)
df['status'].replace({'Out of business- confirmed':'0'}, inplace=True)
df['status'].replace({'Repaid early- Released':'2'}, inplace=True)
df['status'].replace({'Out of business- written off':'0'}, inplace=True)
df['status'].replace({'Repaid early- Released 10/2018':'2'}, inplace=True)
df['status'].replace({'Released 10/2017':'2'}, inplace=True)
df['status'].replace({'Repaid 1/8/2015':'2'}, inplace=True)
df['status'].replace({'Repaid early released 6/18/2018':'2'}, inplace=True)
df['status'].replace({'Out of Business- repaid':'2'}, inplace=True)
df['status'].replace({'Released 12/20/2018':'2'}, inplace=True)
df['status'].replace({'Compliance complete':'2'}, inplace=True)
df['status'].replace({'Company sold to Leggett & Platt':'0'}, inplace=True)
df['status'].replace({'Repaid 11/3/2017':'2'}, inplace=True)
df['status'].replace({'Active- Loan repaid but still w/in 10 years':'2'}, inplace=True)
df['status'].replace({'Repaid 12/31/2017':'2'}, inplace=True)
df['status'].replace({'Released 9/2/2015':'2'}, inplace=True)
df['status'].replace({'Repaid early- Released 10/2013':'2'}, inplace=True)
df['status'].replace({'Out of Business- Confirmed':'0'}, inplace=True)

df['status'].value_counts()

1    1566
2     492
0     109
Name: status, dtype: int64

In [8]:
## Drop Unnecessary Columns

In [9]:
df.drop(columns=['counties', 'zipcode2', 'town_index', 'planning_region', 'geo_location', 'count',
                 'location1', 'part_time_630', 'full_time_630', 'original_pt_job_creation',
                'existing_pt_jobs', 'original_ft_job_creation', 'existing_ft_jobs', 'forgiveness',
                'penalty', 'contract_execution_date', 'min_wom_vet_int',
                'business_industry', 'county_1', 'municipality', 'company_address',
                 'company_name', 'fiscal_year', 'zip_code'], inplace=True)
df.head(5)

,state,naics_code,grant_amount,loan_amount,tot_assistance,total_project_cost,amount_leveraged,funding_source,stat_ref,p_ft_job_retention,p_ft_job_creation,p_pt_job_retention,p_pt_job_creation,actual_jobs,job_obligation_status,status
0,CT,621210,25238.00,0.00,25238.00,50476.00,25238.00,Small Business Express Program,CGS Sec. 32-7g,0,1,0,0,9,Met,1
1,CT,238110,100000.00,0.00,100000.00,200000.00,100000.00,Small Business Express Program,CGS Sec. 32-7g,2,0,0,0,10,Met,1
2,NH,441110,100000.00,0.00,100000.00,533000.00,433000.00,Small Business Express Program,CGS Sec. 32-7g,46,2,0,0,50,Met,1
3,CT,238220,0.00,100000.00,100000.00,100000.00,0.00,Small Business Express Program,CGS Sec. 32-7g,1,5,0,0,0,Not Met,1
4,CT,454390,100000.00,300000.00,400000.00,2931476.00,2531476.00,Small Business Express Program,CGS Sec. 32-7g,17,5,0,0,22,Met,1


## Add new derived columns

In [12]:
#We have a few rows with this See EXP column. Need to remove those
df = df[df['p_ft_job_creation'].str.contains("See EXP")==False]
df = df[df['p_ft_job_creation'].str.contains("-")==False]

#Convert projected jobs to int so they can be added together
df['projected_jobs_total'] = df['p_ft_job_creation'].astype(float) + df['p_pt_job_creation'].astype(float)

In [ ]:
df

In [ ]:
#df['job_obligation_status'] = df['job_obligation_status'].replace(['Met'],'2')
# Adding new columns
df_suc['project_type'] = '2' #Success = 2
df_can['project_type'] = '0' #Cancelled = 0
df_wip['project_type'] = '1' #WIP = 1

In [ ]:
# Make a copy of the df
X = df.copy()

#Drop our response column from the X df
X = X.drop(columns='project_type')

#Need this for labels later
X_labels = X

#Create our y/response df
y = df['project_type']

In [ ]:
#One hot encode X
# create an object of the OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

X = ohe.fit_transform(X)

In [ ]:
#Scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
#Create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#Remember to always work on the train set

In [ ]:
#Create and fit model
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

#Make prediction and check accuracy score
preds = tree_clf.predict(X_test)
acc_score = accuracy_score(preds, y_test)
print('Accuracy=%s' % (acc_score))

In [ ]:
#Grid Search

from sklearn.model_selection import GridSearchCV

#Create dictionary
param_grid = {'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2,3,4,5, 6, 7, 8]}

#Create GridSearch
grid_search_cv = GridSearchCV(tree_clf, param_grid, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)

#Print out best parameters
print(grid_search_cv.best_params_)

print("The best parameters are: ", "max_depth: 2, max_leaf_nodes: 2, min_samples_split: 2")

In [ ]:
from sklearn.tree import export_text

tree_clf2 = DecisionTreeClassifier(max_depth=6, max_leaf_nodes = 7, min_samples_split = 8)
tree_clf2.fit(X, y)
cols = ohe.get_feature_names(input_features=X_labels.columns)
print(export_text(tree_clf2))